# Transformer Based Model for Chat-Bot
Using Pytorch
  

In [1]:
import warnings
warnings.filterwarnings('ignore')

### Import Statements

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import re

### Set Huperparameters for model

- MAX_LEN = 40: The maximum length of input/output sequences (in tokens) for the model.
- BATCH_SIZE = 64: The number of training samples processed in one forward/backward pass.
- NUM_HEADS = 8: The number of attention heads in the multi-head attention mechanism.
- D_MODEL = 512: The dimensionality of the model’s hidden layer representations.
- FFN_UNITS = 2048: The number of units in the feed-forward neural network after each attention layer.
- DROPOUT = 0.1: The fraction of units to drop during training to prevent overfitting.
- NUM_LAYERS = 4: The number of layers in the encoder and decoder of the Transformer.
- EPOCHS = 300: The number of full passes through the training dataset during training.
- VOCAB_SIZE = 8000: The number of unique tokens in the model's vocabulary.


In [3]:
# Hyperparameters
MAX_LEN = 40
BATCH_SIZE = 64
NUM_HEADS = 8
D_MODEL = 512
FFN_UNITS = 2048
DROPOUT = 0.1
NUM_LAYERS = 4
EPOCHS = 300
VOCAB_SIZE = 8000

#### Setup decide for training

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


#### load CSV file

In [5]:
file_path = 'data/reminiscences_of_a_stock_operator_qa.csv'
data = pd.read_csv(file_path, sep='\t')

In [6]:
questions = data['question'].tolist()
answers = data['answer'].tolist()

### Custom Tokenizer to keepp track of vocab and word to index

In [7]:
class CustomTokenizer:
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.word2idx = {"<pad>": 0, "<start>": 1, "<end>": 2, "<unk>": 3}
        self.idx2word = {0: "<pad>", 1: "<start>", 2: "<end>", 3: "<unk>"}
        self.word_count = {}

    def fit_on_texts(self, texts):
        for text in texts:
            for word in text.split():
                self.word_count[word] = self.word_count.get(word, 0) + 1
        sorted_vocab = sorted(self.word_count.items(), key=lambda x: x[1], reverse=True)[:self.vocab_size - 4]
        for idx, (word, _) in enumerate(sorted_vocab, start=4):
            self.word2idx[word] = idx
            self.idx2word[idx] = word

    def texts_to_sequences(self, texts):
        sequences = []
        for text in texts:
            seq = [self.word2idx.get(word, self.word2idx["<unk>"]) for word in text.split()]
            sequences.append(seq)
        return sequences

    def sequences_to_texts(self, sequences):
        texts = []
        for seq in sequences:
            text = " ".join([self.idx2word.get(idx, "<unk>") for idx in seq])
            texts.append(text)
        return texts


In [8]:
tokenizer = CustomTokenizer(VOCAB_SIZE)
tokenizer.fit_on_texts(questions + answers)

### Initialize the dataset by converting the questions and answers to sequences of integers.

In [9]:
class ChatDataset(Dataset):
    def __init__(self, questions, answers, tokenizer, max_len):
        self.questions = tokenizer.texts_to_sequences(questions)
        self.answers = tokenizer.texts_to_sequences(answers)
        self.max_len = max_len

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        answer = self.answers[idx]

        question = [1] + question[:self.max_len - 2] + [2]
        answer = [1] + answer[:self.max_len - 2] + [2]

        question = question + [0] * (self.max_len - len(question))
        answer = answer + [0] * (self.max_len - len(answer))

        return torch.tensor(question), torch.tensor(answer)

### Compute the scaled dot-product attention.

In [10]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = torch.matmul(q, k.transpose(-2, -1))
    dk = q.size(-1)
    scaled_attention_logits = matmul_qk / torch.sqrt(torch.tensor(dk, dtype=torch.float32, device=q.device))
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)
    attention_weights = F.softmax(scaled_attention_logits, dim=-1)
    output = torch.matmul(attention_weights, v)
    return output

### Multi Head Atention 

In [11]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.depth = d_model // num_heads

        self.wq = nn.Linear(d_model, d_model)
        self.wk = nn.Linear(d_model, d_model)
        self.wv = nn.Linear(d_model, d_model)

        self.dense = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask):
        batch_size = q.size(0)
        q = self.wq(q).view(batch_size, -1, self.num_heads, self.depth).transpose(1, 2)
        k = self.wk(k).view(batch_size, -1, self.num_heads, self.depth).transpose(1, 2)
        v = self.wv(v).view(batch_size, -1, self.num_heads, self.depth).transpose(1, 2)

        attention = scaled_dot_product_attention(q, k, v, mask)
        attention = attention.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        return self.dense(attention)

### Initialize the feed-forward network with two linear layers, ReLU activation, and dropout regularization

In [12]:
class FeedForwardNetwork(nn.Module):
    def __init__(self, d_model, ffn_units, dropout_rate):
        super(FeedForwardNetwork, self).__init__()
        self.linear1 = nn.Linear(d_model, ffn_units)
        self.linear2 = nn.Linear(ffn_units, d_model)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.dropout(x)
        return self.linear2(x)

### Encoder: Multi Head Attention with Layer normalization

In [13]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ffn_units, dropout_rate):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForwardNetwork(d_model, ffn_units, dropout_rate)
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, mask):
        attn_output = self.attention(x, x, x, mask)
        out1 = self.layernorm1(x + self.dropout(attn_output))
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + self.dropout(ffn_output))

### Decoder: With look ahead mask and Cross attention

In [14]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, ffn_units, dropout_rate):
        super(DecoderLayer, self).__init__()
        self.attention1 = MultiHeadAttention(d_model, num_heads)
        self.attention2 = MultiHeadAttention(d_model, num_heads)
        self.ffn = FeedForwardNetwork(d_model, ffn_units, dropout_rate)
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.layernorm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, enc_output, look_ahead_mask, padding_mask):
        attn1 = self.attention1(x, x, x, look_ahead_mask)
        out1 = self.layernorm1(x + self.dropout(attn1))

        attn2 = self.attention2(out1, enc_output, enc_output, padding_mask)
        out2 = self.layernorm2(out1 + self.dropout(attn2))

        ffn_output = self.ffn(out2)
        return self.layernorm3(out2 + self.dropout(ffn_output))

In [15]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len):
        super(PositionalEncoding, self).__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-np.log(10000.0) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.encoding[:, :x.size(1), :].to(x.device)

In [16]:
class Transformer(nn.Module):
    def __init__(self, vocab_size, d_model, num_heads, ffn_units, num_layers, dropout_rate, max_len):
        super(Transformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_len)

        # Encoder
        self.encoder_layers = nn.ModuleList([
            EncoderLayer(d_model, num_heads, ffn_units, dropout_rate)
            for _ in range(num_layers)
        ])

        # Decoder
        self.decoder_layers = nn.ModuleList([
            DecoderLayer(d_model, num_heads, ffn_units, dropout_rate)
            for _ in range(num_layers)
        ])

        self.fc = nn.Linear(d_model, vocab_size)

    def create_look_ahead_mask(self, size):
        mask = torch.triu(torch.ones(size, size), diagonal=1)
        return mask == 1

    def forward(self, encoder_input, decoder_input, encoder_mask=None, decoder_mask=None):
        # Encoder
        encoder_embedded = self.embedding(encoder_input)
        encoder_embedded = self.positional_encoding(encoder_embedded)

        encoder_output = encoder_embedded
        for layer in self.encoder_layers:
            encoder_output = layer(encoder_output, encoder_mask)

        # Decoder
        decoder_embedded = self.embedding(decoder_input)
        decoder_embedded = self.positional_encoding(decoder_embedded)

        look_ahead_mask = self.create_look_ahead_mask(decoder_input.size(1)).to(decoder_input.device)

        decoder_output = decoder_embedded
        for layer in self.decoder_layers:
            decoder_output = layer(decoder_output, encoder_output, look_ahead_mask, encoder_mask)

        return self.fc(decoder_output)

In [17]:
model = Transformer(VOCAB_SIZE, D_MODEL, NUM_HEADS, FFN_UNITS, NUM_LAYERS, DROPOUT, MAX_LEN)
model = model.to(device)

In [18]:
dataset = ChatDataset(questions, answers, tokenizer, MAX_LEN)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

### Optimizer and loss functions

In [19]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

### Model Training

In [20]:
# Training Loop with Loss and Accuracy
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    total_accuracy = 0
    total_tokens = 0

    for questions_batch, answers_batch in dataloader:
        questions_batch = questions_batch.to(device)
        decoder_input = answers_batch[:, :-1].to(device)  # Input for decoder
        target = answers_batch[:, 1:].to(device)  # Target for training

        optimizer.zero_grad()

        output = model(questions_batch, decoder_input)
        loss = criterion(output.view(-1, VOCAB_SIZE), target.view(-1))

        # Backpropagation
        loss.backward()
        optimizer.step()

        # Calculate accuracy
        predicted_tokens = torch.argmax(output, dim=-1)
        correct_tokens = (predicted_tokens == target).sum().item()
        total_accuracy += correct_tokens
        total_tokens += target.numel()

        total_loss += loss.item()

    epoch_loss = total_loss / len(dataloader)
    epoch_accuracy = total_accuracy / total_tokens

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.4f}")

Epoch 1/300, Loss: 7.3356, Accuracy: 0.1414
Epoch 2/300, Loss: 6.3788, Accuracy: 0.1999
Epoch 3/300, Loss: 5.8830, Accuracy: 0.2210
Epoch 4/300, Loss: 5.5429, Accuracy: 0.2354
Epoch 5/300, Loss: 5.2682, Accuracy: 0.2502
Epoch 6/300, Loss: 5.0540, Accuracy: 0.2712
Epoch 7/300, Loss: 4.8352, Accuracy: 0.2894
Epoch 8/300, Loss: 4.6328, Accuracy: 0.3056
Epoch 9/300, Loss: 4.4609, Accuracy: 0.3236
Epoch 10/300, Loss: 4.2691, Accuracy: 0.3409
Epoch 11/300, Loss: 4.0866, Accuracy: 0.3609
Epoch 12/300, Loss: 3.9866, Accuracy: 0.3744
Epoch 13/300, Loss: 3.8690, Accuracy: 0.3898
Epoch 14/300, Loss: 3.6942, Accuracy: 0.4036
Epoch 15/300, Loss: 3.5686, Accuracy: 0.4177
Epoch 16/300, Loss: 3.4420, Accuracy: 0.4329
Epoch 17/300, Loss: 3.3210, Accuracy: 0.4463
Epoch 18/300, Loss: 3.1913, Accuracy: 0.4575
Epoch 19/300, Loss: 3.0824, Accuracy: 0.4709
Epoch 20/300, Loss: 2.9842, Accuracy: 0.4839
Epoch 21/300, Loss: 2.8825, Accuracy: 0.4983
Epoch 22/300, Loss: 2.7703, Accuracy: 0.5101
Epoch 23/300, Loss:

### Export the Model

In [21]:
torch.save(model.state_dict(), 'model/transformer_chatbot_gpu_deco_1.pth')

### Load and Test the model

In [22]:
# Load the Trained Model
model.load_state_dict(torch.load("model/transformer_chatbot_gpu_deco_1.pth"))
model.eval()

# Chat Function
def chat_response(question, tokenizer, model, max_len=40):
    model.eval()
    question_seq = tokenizer.texts_to_sequences([question])[0]
    question_seq = [1] + question_seq[:max_len - 2] + [2]  # Add <start> and <end> tokens
    question_seq = question_seq + [0] * (max_len - len(question_seq))  # Pad to max_len
    question_tensor = torch.tensor([question_seq]).to(device)

    decoder_input = torch.tensor([[1]]).to(device)  # Start token
    response = []

    for _ in range(max_len):
        with torch.no_grad():
            output = model(question_tensor, decoder_input)
        predicted_id = torch.argmax(output[:, -1, :], dim=-1).item()
        if predicted_id == 2:  # End token
            break
        response.append(predicted_id)
        decoder_input = torch.cat([decoder_input, torch.tensor([[predicted_id]]).to(device)], dim=-1)

    return tokenizer.sequences_to_texts([response])[0].replace("<start>", "").replace("<end>", "").strip()

# Interactive Chat Loop
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting chatbot. Goodbye!")
        break
    bot_response = chat_response(user_input, tokenizer, model)
    print(f"Bot: {bot_response}")


You:  what was your first job in finance?


Bot: My first job was as a quotation-board boy at a stock brokerage firm. I was quick with numbers and excelled at mental arithmetic, skills that proved invaluable later in my career.


You:  what prompted you to leave bucket shop?


Bot: The bucket shops became unhappy with my consistent success, and they began to actively restrict my trades. It was time to seek a more challenging and professional trading environment.


You:  why did you to leave bucket shop?


...

Bot: My understanding of market psychology played a critical role in the influence of the influence of fear and greed from their own judgments and greed from their mistakes. Recognizing the influence of others.


You:  what did you learn from your biggest loss?


Bot: My biggest loss reinforced the importance of anticipating market shifts and managing risk effectively. It also highlighted the need for maintaining emotional control and a consistent approach to trading.


You:  exit


Exiting chatbot. Goodbye!
